In [ ]:
import pandas as pd
import numpy as np
import os
import datetime
import sys
import seaborn as sns
import matplotlib.pyplot as plt

from data_pipeline.score import field_names
from data_pipeline.comparison_tool.src import utils

pd.options.display.float_format = "{:,.3f}".format
%load_ext lab_black

# Comparator definition comparison

This notebook answers a few questions:
1. How many tracts are flagged and what's the size of overlap by comparator?
2. What are the demographics of each set of tracts by "category" of score (CEJST but not comparator, comparator but not CEJST, CEJST and comparator)?
3. What are the overall demographics of ALL comparator vs ALL CEJST?

It produces a single Excel file of the stats listed, but is interactive even after run-time. This notebook focuses on 1:1 comparison. It can be pointed in the YAML to either a simple output (tract and boolean for highlight) or to the output from an ETL.

In [ ]:
## These are parameters and get overridden by the "injected parameters" cell below
ADDITIONAL_DEMO_COLUMNS = []
COMPARATOR_COLUMN = None
COMPARATOR_FILE = None
DEMOGRAPHIC_COLUMNS = []
DEMOGRAPHIC_FILE = None
OUTPUT_DATA_PATH = None
SCORE_FILE = None
OTHER_COMPARATOR_COLUMNS = None
OUTPUT_NAME = None
KEEP_MISSING_VALUES_FOR_SEGMENTATION = True

In [ ]:
## These are constants for all runs
GEOID_COLUMN = field_names.GEOID_TRACT_FIELD
SCORE_COLUMN = field_names.SCORE_M_COMMUNITIES
TOTAL_POPULATION_COLUMN = field_names.TOTAL_POP_FIELD

__Date and time of last run__

In [ ]:
datetime.datetime.now()

__Congfigure output (autocreated)__

In [ ]:
OUTPUT_EXCEL = os.path.join(
    OUTPUT_DATA_PATH,
    f"{OUTPUT_NAME}__{datetime.datetime.now().strftime('%Y-%m-%d')}.xlsx",
)

__Validate new data__

In [ ]:
utils.validate_new_data(file_path=COMPARATOR_FILE, score_col=COMPARATOR_COLUMN)

__Read in data__

In [ ]:
comparator_cols = (
    [COMPARATOR_COLUMN] + OTHER_COMPARATOR_COLUMNS
    if OTHER_COMPARATOR_COLUMNS
    else [COMPARATOR_COLUMN]
)

# papermill_description=Loading_data
joined_df = pd.concat(
    [
        utils.read_file(
            file_path=SCORE_FILE,
            columns=[TOTAL_POPULATION_COLUMN, SCORE_COLUMN]
            + ADDITIONAL_DEMO_COLUMNS,
            geoid=GEOID_COLUMN,
        ),
        utils.read_file(
            file_path=COMPARATOR_FILE,
            columns=comparator_cols,
            geoid=GEOID_COLUMN,
        ),
        utils.read_file(
            file_path=DEMOGRAPHIC_FILE,
            columns=DEMOGRAPHIC_COLUMNS,
            geoid=GEOID_COLUMN,
        ),
    ],
    axis=1,
).reset_index()

## High-level summary

What *shares* of tracts and population highlighted by the comparator are covered by CEJST?

In [ ]:
# papermill_description=Summary_stats
population_df = utils.produce_summary_stats(
    joined_df=joined_df,
    comparator_column=COMPARATOR_COLUMN,
    score_column=SCORE_COLUMN,
    population_column=TOTAL_POPULATION_COLUMN,
    geoid_column=GEOID_COLUMN,
)
population_df

## Tract-level stats

First, this walks through overall stats for disadvantaged communities under the comparator definition and under the CEJST's definition. Next, this walks through stats by group (e.g., CEJST and not comparator). This is at the tract level, so the average across tracts, where tracts are not population-weighted. 

In [ ]:
# papermill_description=Tract_stats
tract_level_by_identification_df = pd.concat(
    [
        utils.get_demo_series(
            grouping_column=COMPARATOR_COLUMN,
            joined_df=joined_df,
            demo_columns=ADDITIONAL_DEMO_COLUMNS + DEMOGRAPHIC_COLUMNS,
        ),
        utils.get_demo_series(
            grouping_column=SCORE_COLUMN,
            joined_df=joined_df,
            demo_columns=ADDITIONAL_DEMO_COLUMNS + DEMOGRAPHIC_COLUMNS,
        ),
    ],
    axis=1,
)

tract_level_by_identification_df

In [ ]:
plt.figure(figsize=(11, 11))
sns.barplot(
    y="Variable",
    x="Avg in tracts",
    hue="Definition",
    data=tract_level_by_identification_df.sort_values(
        by=COMPARATOR_COLUMN, ascending=False
    )
    .stack()
    .reset_index()
    .rename(
        columns={
            "level_0": "Variable",
            "level_1": "Definition",
            0: "Avg in tracts",
        }
    ),
    palette="Blues",
)
plt.xlim(0, 1)
plt.title("Tract level averages by identification strategy")
plt.savefig(
    os.path.join(OUTPUT_DATA_PATH, "tract_lvl_avg.jpg"), bbox_inches="tight"
)

In [ ]:
# papermill_description=Tract_stats_grouped
tract_level_by_grouping_df = utils.get_tract_level_grouping(
    joined_df=joined_df,
    score_column=SCORE_COLUMN,
    comparator_column=COMPARATOR_COLUMN,
    demo_columns=ADDITIONAL_DEMO_COLUMNS + DEMOGRAPHIC_COLUMNS,
    keep_missing_values=KEEP_MISSING_VALUES_FOR_SEGMENTATION,
)

tract_level_by_grouping_formatted_df = utils.format_multi_index_for_excel(
    df=tract_level_by_grouping_df
)

In [ ]:
tract_level_by_grouping_formatted_df

## Population-weighted stats

In [ ]:
# papermill_description=Population_stats
population_weighted_stats_df = pd.concat(
    [
        utils.construct_weighted_statistics(
            input_df=joined_df,
            weighting_column=COMPARATOR_COLUMN,
            demographic_columns=DEMOGRAPHIC_COLUMNS + ADDITIONAL_DEMO_COLUMNS,
            population_column=TOTAL_POPULATION_COLUMN,
        ),
        utils.construct_weighted_statistics(
            input_df=joined_df,
            weighting_column=SCORE_COLUMN,
            demographic_columns=DEMOGRAPHIC_COLUMNS + ADDITIONAL_DEMO_COLUMNS,
            population_column=TOTAL_POPULATION_COLUMN,
        ),
    ],
    axis=1,
)

In [ ]:
population_weighted_stats_df

## Final information about overlap

In [ ]:
comparator_and_cejst_proportion_series, states = utils.get_final_summary_info(
    population=population_df,
    comparator_file=COMPARATOR_FILE,
    geoid_col=GEOID_COLUMN,
)

In [ ]:
states_text = "States included in comparator: " + states
states_text

## Print to excel

In [ ]:
# papermill_description=Writing_excel
utils.write_single_comparison_excel(
    output_excel=OUTPUT_EXCEL,
    population_df=population_df,
    tract_level_by_identification_df=tract_level_by_identification_df,
    population_weighted_stats_df=population_weighted_stats_df,
    tract_level_by_grouping_formatted_df=tract_level_by_grouping_formatted_df,
    comparator_and_cejst_proportion_series=comparator_and_cejst_proportion_series,
    states_text=states_text,
)